In [0]:
from pyspark.sql.types import StructType, StringType, DoubleType, IntegerType

# Define schema based on title.ratings.csv
ratings_schema = StructType() \
    .add("tconst", StringType()) \
    .add("averageRating", DoubleType()) \
    .add("numVotes", IntegerType())

# Read stream
ratings_stream_df = spark.readStream \
    .option("header", True) \
    .schema(ratings_schema) \
    .option("maxFilesPerTrigger", 1) \
    .csv("/FileStore/stream_input_ratings/")

In [0]:
#Write to Delta Table
ratings_output_path = "/FileStore/delta/ratings_streamed"

ratings_query = ratings_stream_df.writeStream \
    .format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", "/FileStore/checkpoints/ratings_streamed") \
    .start(ratings_output_path)

---------------------------------------------------------------------------
UnsupportedOperationException             Traceback (most recent call last)
File <command-8064372585454975>, line 8
      1 #Write to Delta Table
      2 ratings_output_path = "/FileStore/delta/ratings_streamed"
      4 ratings_query = ratings_stream_df.writeStream \
      5     .format("delta") \
      6     .outputMode("append") \
      7     .option("checkpointLocation", "/FileStore/checkpoints/ratings_streamed") \
----> 8     .start(ratings_output_path)

File /databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/streaming/readwriter.py:648, in DataStreamWriter.start(self, path, format, outputMode, partitionBy, queryName, **options)
    639 def start(
    640     self,
    641     path: Optional[str] = None,
   (...)
    646     **options: "OptionalPrimitiveType",
    647 ) -> StreamingQuery:
--> 648     return self._start_internal(
    649         path=path,
    650         tableName=None,
   

In [0]:
%sql
CREATE VOLUME main.imdb.stream_output;

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-8064372585454976>, line 1
----> 1 get_ipython().run_cell_magic('sql', '', 'CREATE VOLUME main.imdb.stream_output;\n')

File /databricks/python/lib/python3.11/site-packages/IPython/core/interactiveshell.py:2541, in InteractiveShell.run_cell_magic(self, magic_name, line, cell)
   2539 with self.builtin_trap:
   2540     args = (magic_arg_s, cell)
-> 2541     result = fn(*args, **kwargs)
   2543 # The code below prevents the output from being displayed
   2544 # when using magics with decorator @output_can_be_silenced
   2545 # when the last Python token in the expression is a ';'.
   2546 if getattr(fn, magic.MAGIC_OUTPUT_CAN_BE_SILENCED, False):

File /databricks/python_shell/lib/dbruntime/sql_magic/sql_magic.py:147, in SqlMagic.sql(self, line, cell)
    143     raise Exception(
    144         "Cannot run %sql command beca

In [0]:
%sql
SHOW CATALOGS;

catalog
samples
system
workspace


In [0]:
%sql
-- Step 1: Create schema
CREATE SCHEMA IF NOT EXISTS workspace.imdb;



In [0]:
%sql
-- Step 2: Create volume to hold streaming data
CREATE VOLUME IF NOT EXISTS workspace.imdb.stream_input;
CREATE VOLUME IF NOT EXISTS workspace.imdb.stream_output;

In [0]:
from pyspark.sql.types import StructType, StringType, IntegerType

schema = StructType() \
    .add("tconst", StringType()) \
    .add("averageRating", StringType()) \
    .add("numVotes", IntegerType())

ratings_stream_df = spark.readStream \
    .schema(schema) \
    .option("maxFilesPerTrigger", 1) \
    .csv("/Volumes/workspace/imdb/stream_input")



In [0]:
output_path = "/Volumes/workspace/imdb/stream_output/ratings_delta"
checkpoint_path = "/Volumes/workspace/imdb/stream_output/ratings_checkpoints"

ratings_query = ratings_stream_df.writeStream \
    .format("delta") \
    .outputMode("append") \
    .trigger(once=True) \
    .option("checkpointLocation", checkpoint_path) \
    .start(output_path)

In [0]:
%sql

DESCRIBE HISTORY delta.`/Volumes/workspace/imdb/stream_output/ratings_delta/`;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
1,2025-06-23T19:52:29.000Z,4022193295141886,nagendra.yadav22796@gmail.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> 18859422-580b-47dd-8109-dd22dfd11ee2, epochId -> 0, statsOnLoad -> false)",null,null,0623-192802-plvfcz0m-v2n,0,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 1579910, numOutputBytes -> 11916043, numAddedFiles -> 1)",null,Databricks-Runtime/16.4.x-aarch64-photon-scala2.12
0,2025-06-23T19:52:22.000Z,4022193295141886,nagendra.yadav22796@gmail.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> 18859422-580b-47dd-8109-dd22dfd11ee2, epochId -> -1, statsOnLoad -> false)",null,null,0623-192802-plvfcz0m-v2n,null,WriteSerializable,true,Map(),null,Databricks-Runtime/16.4.x-aarch64-photon-scala2.12


In [0]:
%sql
SHOW VOLUMES IN workspace.imdb;

database,volume_name
imdb,stream_input
imdb,stream_output


In [0]:
ratings_delta_df = spark.read.format("delta").load(output_path)
ratings_delta_df.show(5)

+---------+-------------+--------+
|   tconst|averageRating|numVotes|
+---------+-------------+--------+
|tt0000001|          5.7|    2163|
|tt0000002|          5.5|     296|
|tt0000003|          6.5|    2217|
|tt0000004|          5.3|     189|
|tt0000005|          6.2|    2955|
+---------+-------------+--------+
only showing top 5 rows


In [0]:
ratings_delta_df.createOrReplaceTempView("ratings_delta")
spark.sql("SELECT * FROM ratings_delta LIMIT 5").show()

+---------+-------------+--------+
|   tconst|averageRating|numVotes|
+---------+-------------+--------+
|tt0000001|          5.7|    2163|
|tt0000002|          5.5|     296|
|tt0000003|          6.5|    2217|
|tt0000004|          5.3|     189|
|tt0000005|          6.2|    2955|
+---------+-------------+--------+



In [0]:
%sql
SELECT * FROM delta.`/Volumes/workspace/imdb/stream_output/ratings_delta/` LIMIT 10

tconst,averageRating,numVotes
tt0000001,5.7,2163
tt0000002,5.5,296
tt0000003,6.5,2217
tt0000004,5.3,189
tt0000005,6.2,2955
tt0000006,5.0,213
tt0000007,5.3,913
tt0000008,5.4,2306
tt0000009,5.4,226
tt0000010,6.8,7993


In [0]:
df = spark.read.option("header", True).csv("/Volumes/workspace/imdb/stream_input/title.ratings.csv")

from pyspark.sql.types import StructType, StringType, DoubleType

# Define schema manually
schema = StructType() \
    .add("userId", StringType()) \
    .add("movieId", StringType()) \
    .add("rating", DoubleType()) \
    .add("timestamp", StringType())

# Read CSV without header
df = spark.read \
    .option("header", False) \
    .schema(schema) \
    .csv("/Volumes/workspace/imdb/stream_input/title.ratings.csv")



In [0]:
# Write to Delta
df.write.format("delta") \
    .mode("overwrite") \
    .save("/Volumes/workspace/imdb/delta/")

In [0]:
%sql
DESCRIBE HISTORY delta.`/Volumes/workspace/imdb/delta/`

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
0,2025-06-23T20:30:26.000Z,4022193295141886,nagendra.yadav22796@gmail.com,WRITE,"Map(mode -> Overwrite, statsOnLoad -> false, partitionBy -> [])",null,null,0623-192802-plvfcz0m-v2n,null,WriteSerializable,false,"Map(numFiles -> 1, numRemovedFiles -> 0, numRemovedBytes -> 0, numOutputRows -> 1579909, numOutputBytes -> 11961826)",null,Databricks-Runtime/16.4.x-aarch64-photon-scala2.12


In [0]:
df_filtered = df.filter("rating >= 4")

df_filtered.write.format("delta") \
    .mode("overwrite") \
    .save("/Volumes/workspace/imdb/delta")

In [0]:
%sql
DESCRIBE HISTORY delta.`/Volumes/workspace/imdb/delta`

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
1,2025-06-23T20:33:59.000Z,4022193295141886,nagendra.yadav22796@gmail.com,WRITE,"Map(mode -> Overwrite, statsOnLoad -> false, partitionBy -> [])",null,null,0623-192802-plvfcz0m-v2n,0,WriteSerializable,false,"Map(numFiles -> 1, numRemovedFiles -> 1, numRemovedBytes -> 11934530, numOutputRows -> 1579910, numOutputBytes -> 11935609)",null,Databricks-Runtime/16.4.x-aarch64-photon-scala2.12
0,2025-06-23T20:33:45.000Z,4022193295141886,nagendra.yadav22796@gmail.com,WRITE,"Map(mode -> Overwrite, statsOnLoad -> false, partitionBy -> [])",null,null,0623-192802-plvfcz0m-v2n,null,WriteSerializable,false,"Map(numFiles -> 1, numRemovedFiles -> 0, numRemovedBytes -> 0, numOutputRows -> 1579910, numOutputBytes -> 11934530)",null,Databricks-Runtime/16.4.x-aarch64-photon-scala2.12


In [0]:
%sql
SELECT * FROM delta.`/Volumes/workspace/imdb/delta`
VERSION AS OF 0

userId,movieId,rating,timestamp
tt0000001,5.7,2163.0,null
tt0000002,5.5,296.0,null
tt0000003,6.5,2217.0,null
tt0000004,5.3,189.0,null
tt0000005,6.2,2955.0,null
tt0000006,5.0,213.0,null
tt0000007,5.3,913.0,null
tt0000008,5.4,2306.0,null
tt0000009,5.4,226.0,null
tt0000010,6.8,7993.0,null
